# Lecture 3 — Jupyter Notebooks for Analysis (Kaggle Dataset Example: Netflix Movies & TV Shows)

**Dataset:** Kaggle “Netflix Movies and TV Shows” (`netflix_titles.csv`)  
Dataset page: https://www.kaggle.com/datasets/shivamb/netflix-shows

**Notebook skills highlighted**
- Markdown narrative + code outputs (analysis as a report)
- Reproducibility: `Kernel → Restart & Run All`
- Relative paths + simple project structure
- Handling missing values + date parsing
- Clean, labeled plots + short interpretations

---

## 0) Recommended folder structure

```
lecture3_netflix/
  notebooks/
    L3_netflix_notebook.ipynb
  data/
    netflix/
      netflix_titles.csv
```

> Put the CSV in `data/netflix/` before running the load cell.


## 1) Getting the dataset (two options)

### Option A — Manual download (best for lecture day)
1. Open the Kaggle dataset page.
2. Download the dataset ZIP.
3. Extract `netflix_titles.csv` into: `data/netflix/`.

### Option B — Kaggle API download (reproducible)
1. Create a Kaggle API token (`kaggle.json`) from Kaggle Account settings.
2. Put it in:
   - macOS/Linux: `~/.kaggle/kaggle.json`
   - Windows: `C:\Users\<you>\.kaggle\kaggle.json`
3. macOS/Linux permissions: `chmod 600 ~/.kaggle/kaggle.json`
4. Run the download cell below.

> **Class tip:** Option A avoids account setup issues in-class. Option B is great to demonstrate reproducible data access.


In [3]:
import os
from pathlib import Path
import subprocess
import zipfile

DATA_DIR = Path("data/netflix")
DATA_DIR.mkdir(parents=True, exist_ok=True)

csv_path = DATA_DIR / "netflix_titles.csv"

if csv_path.exists():
    print("✅ Found:", csv_path)
else:
    print("netflix_titles.csv not found. Attempting Kaggle download...")
    print("If this fails, use manual download (Option A).")

    # Ensure kaggle is installed
    try:
        import kaggle  # noqa: F401
    except Exception:
        subprocess.check_call([os.sys.executable, "-m", "pip", "install", "-q", "kaggle"])

    # Download dataset zip
    subprocess.check_call(["kaggle", "datasets", "download", "-d", "shivamb/netflix-shows", "-p", str(DATA_DIR)])

    # Unzip all zips in DATA_DIR
    for z in DATA_DIR.glob("*.zip"):
        print("Unzipping:", z.name)
        with zipfile.ZipFile(z, "r") as zip_ref:
            zip_ref.extractall(DATA_DIR)

    print("✅ Done. Files:", [p.name for p in DATA_DIR.glob('*')])


netflix_titles.csv not found. Attempting Kaggle download...
If this fails, use manual download (Option A).
Could not find kaggle.json. Make sure it's located in C:\Users\emmal\.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


FileNotFoundError: [WinError 2] The system cannot find the file specified

## 2) Imports + settings (reproducibility cell)

A good notebook has a **single place** for imports and global settings.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

SEED = 0 #always set your seed for reproducibility
rng = np.random.default_rng(SEED)

pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

DATA_DIR = Path("data/netflix")
CSV_PATH = DATA_DIR / "netflix_titles.csv"


ModuleNotFoundError: No module named 'matplotlib'

## 3) Load data (relative path)

**Notebook best practice:** do not hard-code absolute paths (`/Users/...`).


In [ ]:
if not CSV_PATH.exists():
    raise FileNotFoundError(
        f"Missing {CSV_PATH}.\n"
        "Download from Kaggle (manual) or run the Kaggle API download cell above."
    )

df = pd.read_csv(CSV_PATH)
df.head()


NameError: name 'CSV_PATH' is not defined

## 4) Quick checks (shape, columns, missingness)

Keep this section short and repeatable.


In [ ]:
df.shape

In [ ]:
df.columns.tolist()

In [ ]:
df.info()

In [ ]:
df.isna().sum().sort_values(ascending=False).head(12)

## 5) Minimal cleaning for EDA

We’ll:
- Parse `date_added` (often missing)
- Create `year_added`
- Keep missing values as-is (but handle them in group-bys)


In [ ]:
# Parse dates (coerce errors -> NaT)
df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")
df["year_added"] = df["date_added"].dt.year

df[["title", "type", "date_added", "year_added"]].head(10)


## 6) EDA questions (write them in Markdown)

1. How many **Movies vs TV Shows**?
2. How has content **added over time** changed?
3. Which **countries** appear most often? (Note: multiple countries per title)
4. What are the most common **ratings**?


In [ ]:

# Q1) Movies vs TV Shows
type_counts = df["type"].value_counts(dropna=False)
type_counts


In [ ]:
plt.figure()
plt.bar(type_counts.index.astype(str), type_counts.values)
plt.xlabel("Type")
plt.ylabel("Count")
plt.title("Netflix titles: Movies vs TV Shows")
plt.show()


In [ ]:
# Q2) Titles added per year (ignoring missing dates)
added_per_year = df.dropna(subset=["year_added"]).groupby("year_added").size()
added_per_year.tail()


In [ ]:
plt.figure()
plt.plot(added_per_year.index, added_per_year.values)
plt.xlabel("Year added")
plt.ylabel("Titles added")
plt.title("Titles added to Netflix by year (from date_added)")
plt.show()


In [ ]:
# Q3) Top countries
# country field can contain multiple countries separated by commas.
countries = (
    df["country"]
    .dropna()
    .str.split(",")
    .explode()
    .str.strip()
)
top_countries = countries.value_counts().head(10)
top_countries


In [ ]:
plt.figure()
plt.barh(top_countries.index[::-1], top_countries.values[::-1])
plt.xlabel("Count")
plt.ylabel("Country")
plt.title("Top 10 countries listed (titles may have multiple countries)")
plt.show()


In [ ]:
# Q4) Ratings distribution
rating_counts = df["rating"].value_counts(dropna=False).head(12)
rating_counts


In [ ]:
plt.figure()
plt.bar(rating_counts.index.astype(str), rating_counts.values)
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Top ratings (first 12)")
plt.xticks(rotation=45, ha="right")
plt.show()


## 7) Interpretation (write in full sentences)

Write **3–6 sentences** interpreting what you found. Example (edit in class):

- The catalog is dominated by Movies relative to TV Shows.
- Additions per year show a clear trend (peaks/declines) that could reflect data coverage or platform strategy.
- The country field often includes multiple countries per title; the top countries represent frequent production origins.
- Ratings are skewed toward a few common categories, suggesting a dominant audience focus.


## 8) Reproducibility check (before submission)

**Rule:** Run `Kernel → Restart & Run All`.  
If anything breaks, you likely have hidden state or missing imports.


In [ ]:
print("If you reached this cell after Restart & Run All, your notebook is reproducible.")


## 9) Export / share

- Export HTML (terminal):
  - `jupyter nbconvert --to html notebooks/L3_netflix_notebook.ipynb`
- For Git: avoid committing huge outputs; keep notebooks clean.
